1. remove useless columns
2. filters from params.yaml

In [34]:
import logging
import sys
import time

import polars as pl
import yaml

logger = logging.getLogger("mr. cleannnnn")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(
    logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
)
logger.addHandler(handler)

with open("../params.yaml", "r") as file:
    params = yaml.safe_load(file)

In [35]:
input_file_path = params["clean"]["input_data_path"]

# read in the complete data frame, this one is read as a LazyFrame since it is the largest, around 3.5GB
df = pl.scan_parquet("../" + input_file_path)

In [36]:
# drop columns that will never be used
df = df.drop(
    [
        "Unnamed: 0",
        "events",
        "spin_dir",
        "spin_rate_deprecated",
        "break_angle_deprecated",
        "break_length_deprecated",
        "game_type",
        "home_team",
        "away_team",
        "des",
        "description",
        "game_year",
        "tfs_deprecated",
        "tfs_zulu_deprecated",
        "home_score",
        "away_score",
        "fielder_2",
        "umpire",
        "sv_id",
        "pitcher.1",
        "fielder_2.1",
        "fielder_3",
        "fielder_4",
        "fielder_5",
        "fielder_6",
        "fielder_7",
        "fielder_8",
        "fielder_9",
        "pitch_name",
        "p_throws",
        "zone",
        "if_fielding_alignment",
        "of_fielding_alignment",
        "inning_topbot",
        "vx0",
        "vy0",
        "vz0",
        "sz_top",
        "sz_bot",
        "effective_speed",
        "pfx_x",
        "pfx_z",
        "plate_x",
        "plate_z",
        "type",
    ]
)

In [37]:
df = df.with_columns(pl.col("game_date").str.to_datetime())
df = df.filter(pl.col("game_date").dt.year() >= params["clean"]["start_year"])

In [38]:
df.collect().describe()


statistic,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,stand,hit_location,bb_type,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,ax,ay,az,hit_distance_sc,launch_speed,launch_angle,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
str,str,str,f64,f64,f64,str,f64,f64,str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""","""4015327""","""4104468""",4.014156e6,4.013927e6,4.013927e6,"""4104468""",4.104468e6,4.104468e6,"""4104468""",933002.0,"""722211""",4.104468e6,4.104468e6,384963.0,769830.0,1.258451e6,4.104468e6,4.104468e6,721538.0,721538.0,4.014137e6,4.014137e6,4.014137e6,1.274528e6,1.276975e6,1.278019e6,3.985171e6,4.007504e6,4.104468e6,4.013927e6,689911.0,991961.0,1.070414e6,996250.0,1.070414e6,1.070414e6,689911.0,4.104468e6,4.104468e6,4.104468e6,4.104468e6,4.104468e6,4.104468e6,4.104468e6,4.104468e6,3.998112e6,4.104436e6,3.914658e6,321999.0,321999.0
"""null_count""","""89141""","""0""",90312.0,90541.0,90541.0,"""0""",0.0,0.0,"""0""",3.171466e6,"""3382257""",0.0,0.0,3.719505e6,3.334638e6,2.846017e6,0.0,0.0,3.38293e6,3.38293e6,90331.0,90331.0,90331.0,2.82994e6,2.827493e6,2.826449e6,119297.0,96964.0,0.0,90541.0,3.414557e6,3.112507e6,3.034054e6,3.108218e6,3.034054e6,3.034054e6,3.414557e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,106356.0,32.0,189810.0,3.782469e6,3.782469e6
"""mean""",null,"""2022-03-07 01:15:27.357406""",88.893968,-0.792029,5.819772,null,610559.127989,608783.5297,null,5.084658,null,0.871729,0.891882,612336.842839,612199.766586,610504.72305,0.981007,4.951493,126.205676,122.08389,-2.248643,26.627403,-23.737127,156.301589,82.685152,16.90322,2249.502047,6.303656,664007.006621,54.196259,0.325589,0.316473,0.326857,0.996127,0.18842,0.148394,3.197546,38.625654,2.896572,2.270819,2.314129,2.386855,2.22963,2.302356,2.314129,176.295041,0.000112,0.000003,69.480058,7.216324
"""std""",null,null,6.048116,1.878,0.535392,null,65037.336598,65426.071778,null,2.632107,null,0.967662,0.828766,64210.062319,64476.277576,65175.23253,0.817915,2.607592,40.736932,42.02047,10.649416,4.000878,8.859137,121.274144,15.132133,32.316204,340.361395,0.490186,61979.644168,0.488984,0.291522,0.377516,0.517845,0.062109,0.391047,0.57191,1.281545,22.649587,1.734488,2.584717,2.680943,2.692018,2.584527,2.59838,2.680943,71.556028,0.028415,0.244075,8.889669,0.994875
"""min""","""CH""","""2019-03-20 00:00:00""",30.1,-5.38,-0.13,"""Abad, Fernando""",282332.0,282332.0,"""L""",1.0,"""fly_ball""",0.0,0.0,405395.0,400085.0,400085.0,0.0,1.0,2.0,2.13,-193.953173,-337.555603,-124.677778,0.0,2.0,-90.0,4.0,0.6,564731.0,47.32,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.776,-1.661,0.85121,0.30587
"""25%""",null,"""2020-09-30 00:00:00""",84.6,-2.12,5.55,null,572228.0,572971.0,null,2.0,null,0.0,0.0,572816.0,572816.0,572191.0,0.0,3.0,101.13,89.37,-11.264601,23.656047,-30.07374,20.0,72.9,-5.0,2095.0,6.0,632192.0,53.87,0.074,0.003,0.0,1.0,0.0,0.0,2.0,19.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,132.0,0.0,-0.067,66.97528,6.644
"""50%""",null,"""2022-05-27 00:00:00""",89.8,-1.5,5.86,null,624413.0,621381.0,null,5.0,null,1.0,1.0,624512.0,624512.0,624413.0,1.0,5.0,125.1,125.01,-2.29824,26.52945,-23.548278,166.0,82.4,19.0,2275.0,6.3,661880.0,54.18,0.231,0.162,0.0,1.0,0.0,0.0,3.0,38.0,3.0,1.0,1.0,2.0,1.0,2.0,1.0,198.0,0.0,-0.018,70.91612,7.27356
"""75%""",null,"""2023-07-29 00:00:00""",93.7,0.99,6.15,null,664058.0,663556.0,null,8.0,null,1.0,2.0,664728.0,664728.0,664058.0,2.0,7.0,153.69,156.61,5.948818,29.562672,-16.015329,245.0,94.6,40.0,2448.0,6.6,718048.0,54.52,0.537,0.588,0.7,1.0,0.0,0.0,4.0,57.0,

In [39]:
print(df.collect().height)


# drop rows with null values in the columns 'pitch_type' and 'pitcher', as they cant be used for training
df = df.drop_nulls(subset=["pitch_type", "pitcher"]).collect()

print(df.height)

4104468
4015327
